<a href="https://colab.research.google.com/github/werowe/HypatiaAcademy/blob/master/stats/ocean_open_meteo_historical.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://open-meteo.com/en/docs/marine-weather-api#latitude=35&longitude=33&past_days=14&start_date=2023-01-01&end_date=2023-12-31&time_mode=time_interval&models=era5_ocean

In [ ]:
https://open-meteo.com/en/docs/marine-weather-api#latitude=35&longitude=33&past_days=14&start_date=2023-01-01&end_date=2023-12-31&time_mode=time_interval&models=era5_ocean

In [ ]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://marine-api.open-meteo.com/v1/marine"
params = {
	"latitude": 35,
	"longitude": 33,
	"hourly": "wave_height",
	"start_date": "2023-01-01",
	"end_date": "2023-12-31",
	"models": "era5_ocean"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_wave_height = hourly.Variables(0).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["wave_height"] = hourly_wave_height

hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe)